Connected to miniML_clean (Python 3.10.14)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
import sys
sys.path.append('../../core/')
import template_matching as tm
import copy

In [2]:
df_eval = pd.read_excel('/Users/verjim/laptop_D_17.01.2022/Schmitz_lab/results/human/data/miniML/events_eval_training2.xlsx')
output_folder = '/Users/verjim/miniML_data/data/model_training_round_2/'

# for extraction of noise stretches
win_size = 750
trace_sampling = 20_000
threshold_low = -1.5

baseline = (win_size/8) * trace_sampling - 0.0005
duration = int(win_size*1/3) * trace_sampling
t_rise = (baseline+(win_size * trace_sampling))/15
t_decay = (baseline+(win_size * trace_sampling))/18

template = tm.make_template(t_rise=t_rise, t_decay=t_decay, baseline=baseline, duration=duration, sampling=trace_sampling)


In [3]:
def find_closest_index (data_file, x, score, label):
    with h5py.File('/Users/verjim/miniML_data/data/' + data_file, 'r') as f:
        # Access trace group and its datasets
        trace_group = f['trace_']
        trace_time = trace_group['trace_time'][:]
        trace_data = trace_group['trace_data'][:]

    closest_index = np.abs(trace_time - x).argmin()
    #x_data = trace_time[closest_index-150 : closest_index+599]
    event_trace = trace_data[closest_index-150 : closest_index+599]
    plt.plot(event_trace)

    return event_trace, score, label

In [4]:
for i in range(len(df_eval)):
    score = df_eval.score[i]
    label = df_eval.label[i]
    if score == 1 and label != 'false negative':
        print('problem with slice ' + df_eval.data_file[i])
        print(i)
    if score == 0 and label != 'false positive':
        print('problem with slice ' + df_eval.data_file[i])
        print(i)

problem with slice 2024_Nov_1122127035.abf1_model_comparison.h5
5
problem with slice 2024_Nov_1122519021.abf5_model_comparison.h5
361
problem with slice 2024_Nov_1123817064.abf4_model_comparison.h5
362


In [5]:
noise_only = ['2024_Nov_1122519021.abf5_model_comparison.h5', \
              '2024_Nov_1123817064.abf4_model_comparison.h5', \
                '2024_Nov_1122427028.abf5_model_comparison.h5']

In [6]:
data_folder = '/Users/verjim/miniML_data/data/'
traces, labels, scores, fn_chan, noise_locs  = [], [], [], [], []
for a, file in enumerate(df_eval.data_file.unique()[:-1]):
    df_fn = df_eval[df_eval.data_file == file].reset_index(drop = True)
    # load traces and event peaks
    with h5py.File(data_folder + df_fn.data_file[0], 'r') as f:
        model_2 = f['model_2']
        event_peaks_2 = model_2['event_peaks2'][:] # X

        trace_group = f['trace_']
        #trace_time = trace_group['trace_time'][:]
        trace_data = trace_group['trace_data'][:]
        fn = trace_group.attrs['filename']
        chan = trace_group.attrs['chan']

    if file == '2024_Nov_1123209020.abf6_model_comparison.h5':
        trace_data = trace_data[:192 * 20_000]
        event_peaks_2 = event_peaks_2[event_peaks_2 < 190]

    if file == '2024_Nov_112021_03_25_0058.abf2_model_comparison.h5':
        trace_data = trace_data[:80 * 20_000]
        event_peaks_2 = event_peaks_2[event_peaks_2 < 80]

    # find false positives
    FPs = (df_fn.x[df_fn.label == 'false positive'].reset_index(drop = True).values * 20_000).astype(int)
    all_events_miniML = (event_peaks_2 * 20_000).astype(int)
    for fp in FPs:
        diffs = all_events_miniML - fp
    
    # find events that are close to false positives
    indx_del = np.where((diffs >= -150) & (diffs <= 150))[0] 
    if fn in noise_only:
        true_nice_events = np.empty(0)
    else:
        true_nice_events = np.delete(all_events_miniML, indx_del) # delete them form the dataset 

    # chose same number of events from each filename
    count_ = int(300/ len(df_eval.data_file.unique()) + 15)
    if len(true_nice_events) <= count_ + 40:
        true_nice_events = true_nice_events
    else:
        true_nice_events = np.random.choice(true_nice_events, count_ + 40, replace = False)

    # flse negatives - same procedure
    FNs = (df_fn.x[df_fn.label == 'false negative'].values * 20_000).astype(int)
    all_event_indx = np.concatenate([FPs, true_nice_events, FNs])
    for k, x in enumerate(all_event_indx):
        #trace_dp = trace_time * 20_000
        if len(trace_data[x - 150 : x + 600]) != 750:
            print('problem with slice ' + df_fn.data_file[0] + 'at ' + str(x))  
        traces.append(trace_data[x - 150 : x + 600])

    # create labels
    FP_labels = ['FP; by model'] * len(FPs)
    FP_score = [0] * len(FPs)

    nice_events_labels = ['nice events'] * len(true_nice_events)
    nice_events_scores = [1] * len(true_nice_events)

    FN_labels = ['missed events'] * len(FNs)
    FN_scores = [1] * len(FNs)

    labels = labels + FP_labels + nice_events_labels + FN_labels
    scores = scores + FP_score + nice_events_scores + FN_scores
    fn_chan = fn_chan + [fn + '_' + chan] * len(FP_score + nice_events_scores + FN_scores)

    # extract noise stretches
    event_free_indices = []
    for k, x in enumerate(sorted(all_event_indx)):
        if k == len(all_event_indx) - 1:
            continue
        start = x + 600
        end = (all_event_indx[k + 1] - 150)
        if end - start > 0:
            event_free_indices.append(np.arange(start, end))

    event_free_indices = np.concatenate(event_free_indices)
    unique_stretches = []
    for ind, i in enumerate(event_free_indices):
        if ind==0:
            unique_stretches.append(i)
            next_possible = i+win_size
        
        if i < next_possible:
            pass
        else:
            unique_stretches.append(i)
            next_possible = i+win_size

    # Extract same number of unique noise stretches from each fn
    if len(unique_stretches) <= count_:
        inds = np.array(unique_stretches)
    else:
        inds = np.random.choice(np.array(unique_stretches), count_, replace=False)

    # Collect noise stretche
    noise_counter = 0
    for location in sorted(inds):
        event = copy.deepcopy(trace_data[location:location+win_size])
        event -= np.mean(event[:int(win_size/10)])

        traces.append(event)
        labels.append('noise')
        scores.append(0)
        fn_chan.append(fn + '_' + chan)
        
        noise_locs.append(location)
        noise_counter += 1
        
traces = np.vstack(traces)

In [7]:
print('nice events :', labels.count('nice events'), 'score 1')
print('missed events :', labels.count('missed events'), 'score 1')
print('FP; by model :', labels.count('FP; by model'), 'score 0')
print('noise :', labels.count('noise'), 'score 0')

print('All events :', len(labels))

nice events : 352 score 1
missed events : 303 score 1
FP; by model : 135 score 0
noise : 384 score 0
All events : 1174


In [8]:
# save the results

with h5py.File(output_folder + file[:-33] + '_all_types_of_events_to_check.h5', 'w') as f:
    f.create_dataset("events", data = traces)
    f.create_dataset("scores", data = scores)
    f.create_dataset("labels", data = labels)
    f.create_dataset("filename and chan", data = fn_chan)

print(f"Everything saved to {output_folder}")

Everything saved to /Users/verjim/miniML_data/data/model_training_round_2/


## Noise estiamtion on specific traces

In [ ]:
trace_name = '2024_Nov_1124117016.abf7_model_comparison.h5' 

data_folder = '/Users/verjim/miniML_data/data/'
traces, labels, scores, fn_chan, noise_locs  = [], [], [], [], []
with h5py.File(data_folder + trace_name, 'r') as f:
    model_2 = f['model_2']
    event_peaks_2 = model_2['event_peaks2'][:] # X

    trace_group = f['trace_']
    #trace_time = trace_group['trace_time'][:]
    trace_data = trace_group['trace_data'][:]
    fn = trace_group.attrs['filename']
    chan = trace_group.attrs['chan']

all_events_miniML = (event_peaks_2 * 20_000).astype(int)

# extract noise stretches
event_free_indices = []
for k, x in enumerate(sorted(all_event_indx)):
    if k == len(all_event_indx) - 1:
        continue
    start = x + 600
    end = (all_event_indx[k + 1] - 150)
    if end - start > 0:
        event_free_indices.append(np.arange(start, end))

event_free_indices = np.concatenate(event_free_indices)
unique_stretches = []
for ind, i in enumerate(event_free_indices):
    if ind==0:
        unique_stretches.append(i)
        next_possible = i+win_size
        
    if i < next_possible:
        pass
    else:
        unique_stretches.append(i)
        next_possible = i+win_size
        
inds = np.array(unique_stretches)
# Extract events
noise_counter = 0
for location in sorted(inds):
    event = copy.deepcopy(trace_data[location:location+win_size])
    event -= np.mean(event[:int(win_size/10)])

    traces.append(event)
    labels.append('noise')
    scores.append(0)
    fn_chan.append(fn_chan)
    
    noise_locs.append(location)
    noise_counter += 1

In [ ]:
concatenated_traces = np.concatenate(traces)

# Plot the results
plt.plot(concatenated_traces)
plt.title('Noise ' + trace_name)
plt.xlabel('Index')
plt.ylabel('Value')
plt.show()

# Calculate the standard deviation
std_dev = np.std(concatenated_traces)
print("Standard Deviation:", std_dev)

In [ ]:
# '2024_Nov_1124117016.abf7_model_comparison.h5' - stdv = 6.3848 of all noise stretches 
# visually the signal looked unstable and it was difficult to tell apart events from nooise - random fluctuations in the signal

# '2024_Nov_1124411036.abf3_model_comparison.h5' - stdv = 6.3095 of all noise stretches
# visually the trace is very stable, looks ideal

